In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Digital_Music_5.json.gz

--2024-08-18 10:51:55--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Digital_Music_5.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19408584 (19M) [application/x-gzip]
Saving to: ‘Digital_Music_5.json.gz’

Digital_Music_5.jso 100%[===================>]  18.51M  --.-KB/s    in 0.1s    

2024-08-18 10:51:55 (124 MB/s) - ‘Digital_Music_5.json.gz’ saved [19408584/19408584]



In [ ]:
!pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00


### Imports

In [ ]:
import numpy as np
import json
import pandas as pd
import gzip
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

### Load the dataset: Digital_Music_5

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Digital_Music_5.json.gz')

In [ ]:
MAX_LEN = 100
BATCH_SIZE = 64
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
NUM_CLASSES = 5
EPOCHS = 15

### PreProcess

In [ ]:
data = df[['reviewText', 'overall']]
data['reviewText'] = data['reviewText'].fillna('')
data['reviewText'] = data['reviewText'].astype(str)

<ipython-input-6-ab92a6827353>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].fillna('')
<ipython-input-6-ab92a6827353>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].astype(str)


### Tokenize

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['reviewText'])
sequences = tokenizer.texts_to_sequences(data['reviewText'])
word_index = tokenizer.word_index

In [ ]:
X = pad_sequences(sequences, maxlen=MAX_LEN)
y = data['overall'] - 1

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### AmazonDataset Class Implementation

In [ ]:
class AmazonDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
      return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

### DataLoaders

In [ ]:
train_dataset = AmazonDataset(X_train, y_train)
val_dataset = AmazonDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Implementing a Custom Sentiment Analysis Model with Recurrent Neural Networks (RNN, GRU, LSTM)

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes, num_layers, model_type='LSTM'):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        if model_type == 'RNN':
            self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        elif model_type == 'GRU':
            self.rnn = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        else:
            self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        h, _ = self.rnn(x)
        out = h[:, -1, :]
        out = self.fc(out)
        return out

### Train the model

In [ ]:
def train_model(model, train_loader, val_loader, epochs):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in tqdm(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_train_loss}")

        # Validation
        model.eval()
        y_pred = []
        y_true = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(y_batch.cpu().numpy())

        accuracy = accuracy_score(y_true, y_pred)
        print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
vocab_size = len(word_index) + 1
for model_type in ['RNN', 'GRU', 'LSTM']:
    for num_layers in [1, 2, 3]:
        print(f"Training {model_type} with {num_layers} layers...")
        model = SentimentRNN(vocab_size, EMBEDDING_DIM, HIDDEN_DIM, NUM_CLASSES, num_layers, model_type)
        train_model(model, train_loader, val_loader, EPOCHS)

Training RNN with 1 layers...


100%|██████████| 2123/2123 [00:13<00:00, 152.81it/s]


Epoch 1/15, Loss: 0.6360953835752063
Validation Accuracy: 0.7968


100%|██████████| 2123/2123 [00:12<00:00, 169.98it/s]


Epoch 2/15, Loss: 0.5841559007293595
Validation Accuracy: 0.8090


...

Epoch 13/15, Loss: 0.15710299157809943
Validation Accuracy: 0.8283


100%|██████████| 2123/2123 [00:25<00:00, 81.76it/s]


Epoch 14/15, Loss: 0.1526015721980063
Validation Accuracy: 0.8320


100%|██████████| 2123/2123 [00:25<00:00, 82.84it/s]


Epoch 15/15, Loss: 0.15037217759231156
Validation Accuracy: 0.8387


### Delete the Data for the Second and Fourth Classes, Train on Classes 1, 3, and 5, and Then Test on Classes 2 and 4

In [ ]:
def preprocess(data):
  sequences = tokenizer.texts_to_sequences(data['reviewText'])
  X = pad_sequences(sequences, maxlen=MAX_LEN)
  y = data['overall'] - 1
  return X, y

In [ ]:
train_data_filtered = data[data['overall'].isin([1, 3, 5])]

test_data_classes_2_and_4 = data[data['overall'].isin([2, 4])]

In [ ]:
X_train_filtered, y_train_filtered = preprocess(train_data_filtered)

X_test_classes_2_and_4, y_test_classes_2_and_4 = preprocess(test_data_classes_2_and_4)

In [ ]:
y_train_filtered = le.fit_transform(y_train_filtered)

In [ ]:
X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train_filtered, y_train_filtered, test_size=0.2, random_state=42)

train_dataset_ = AmazonDataset(X_train_, y_train_)
val_dataset_ = AmazonDataset(X_val_, y_val_)

train_loader_ = DataLoader(train_dataset_, batch_size=BATCH_SIZE, shuffle=True)
val_loader_ = DataLoader(val_dataset_, batch_size=BATCH_SIZE)

In [ ]:
model_type = 'LSTM'
num_layers = 3

model = SentimentRNN(vocab_size, EMBEDDING_DIM, HIDDEN_DIM, 3, num_layers, model_type)

train_model(model, train_loader_, val_loader_, EPOCHS)

100%|██████████| 1811/1811 [00:22<00:00, 82.11it/s]


Epoch 1/15, Loss: 0.21083171971549586
Validation Accuracy: 0.9443


100%|██████████| 1811/1811 [00:22<00:00, 81.60it/s]


Epoch 2/15, Loss: 0.1499315571041972
Validation Accuracy: 0.9542


100%|██████████| 1811/1811 [00:21<00:00, 82.68it/s]


...

Epoch 13/15, Loss: 0.03395439852993963
Validation Accuracy: 0.9595


100%|██████████| 1811/1811 [00:22<00:00, 82.30it/s]


Epoch 14/15, Loss: 0.0331034573501625
Validation Accuracy: 0.9601


100%|██████████| 1811/1811 [00:21<00:00, 82.67it/s]


Epoch 15/15, Loss: 0.03326133815912684
Validation Accuracy: 0.9603


In [ ]:
label_mapping_test = {1: 0, 3: 2}
y_test_classes_2_and_4_mapped = np.array([label_mapping_test[label] for label in y_test_classes_2_and_4])
test_dataset_classes_2_and_4 = AmazonDataset(X_test_classes_2_and_4, y_test_classes_2_and_4_mapped)
test_loader_classes_2_and_4 = DataLoader(test_dataset_classes_2_and_4, batch_size=BATCH_SIZE)

In [ ]:
model.eval()
y_pred_classes_2_and_4 = []

with torch.no_grad():
    for X_batch, y_batch in test_loader_classes_2_and_4:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred_classes_2_and_4.extend(predicted.cpu().numpy())

        # Map the real labels and print the real and predicted labels
        for real_label, pred_label in zip(y_batch.cpu().numpy(), predicted.cpu().numpy()):
            if real_label == 0:
                mapped_real_label = 1
            else:
                mapped_real_label = 3
            if pred_label == 1:
                pred_label = 2
            elif pred_label == 2:
                pred_label = 4
            print(f'Real label: {mapped_real_label}, Predicted label: {pred_label}')

Streaming output truncated to the last 5000 lines.
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 1, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 2
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: 4
Real label: 3, Predicted label: